### Base Sintética de Fraudes (PT)

É criada a coluna **`regular`**, indicando se o registro não contém fraude.  
O CSV gerado será utilizado para a detecção de anomalias.  

---

### Synthetic Fraud Dataset (EN)

A **`regular`** column is created to indicate records without fraud.  
The resulting CSV will be used for the anomaly detection experiment.  

In [ ]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# Parâmetros
n = 10000
fraude_pct = 0.10
phantom_pct = 0.0333
upcoding_pct = 0.0333
duplicidade_pct = 0.0333

np.random.seed(42)

# Função para gerar datas aleatórias
def random_date(start, end):
    return start + timedelta(days=random.randint(0, (end - start).days))

# Criar DataFrame base
df = pd.DataFrame({
    'id_atendimento': range(1, n + 1),
    'beneficiario_id': np.random.randint(1000, 9999, size=n),
    'prestador_id': np.random.randint(100, 999, size=n),
    'data_solicitacao': [random_date(datetime(2023, 1, 1), datetime(2024, 12, 31)) for _ in range(n)],
    'CID_': np.random.choice(['A00', 'B20', 'C34', 'D50', 'E11', 'F32', 'G40', 'H10', 'I10'], size=n),
    'status_saude': np.random.choice([-1, 0, 1], size=n, p=[0.4, 0.4, 0.2]),
    'tipo_atendimento': np.random.choice(['consulta', 'exame', 'cirurgia'], size=n, p=[0.6, 0.3, 0.1]),
    'procedimento_codigo': np.random.randint(10000, 99999, size=n),
    'autorizacao_status': np.random.choice(['aprovado', 'negado', 'pendente'], size=n, p=[0.7, 0.1, 0.2]),
    'data_realizacao': [random_date(datetime(2023, 1, 2), datetime(2025, 1, 1)) for _ in range(n)],
    'valor_pago': np.round(np.random.uniform(50, 1000, size=n), 2),
    'procedimentos_urgencia': np.random.choice(['sim', 'nao'], size=n, p=[0.2, 0.8])
})

# Inicializa colunas de fraude como False
df['phantom_billing'] = False
df['upcoding'] = False
df['duplicidade_pagamento'] = False

# Phantom Billing (valor_pago > 0 e data_realizacao = NaT)
phantom_idx = df.sample(frac=phantom_pct).index
df.loc[phantom_idx, 'data_realizacao'] = pd.NaT
df.loc[phantom_idx, 'phantom_billing'] = True

# Upcoding (status_saude = -1 e procedimentos_urgencia mudado de 'nao' para 'sim')
upcoding_idx = df[
    (df['status_saude'] == -1) & (df['procedimentos_urgencia'] == 'nao')
].sample(frac=upcoding_pct / 0.4).index
df.loc[upcoding_idx, 'procedimentos_urgencia'] = 'sim'
df.loc[upcoding_idx, 'upcoding'] = True

# Duplicidade de cobrança (valor_pago multiplicado entre 3x e 5x)
duplicidade_idx = df.sample(frac=duplicidade_pct).index
df.loc[duplicidade_idx, 'valor_pago'] *= np.random.uniform(3, 5, size=len(duplicidade_idx))
df.loc[duplicidade_idx, 'duplicidade_pagamento'] = True

# Criar coluna 'regular': 'sim' se não houve fraude, 'não' caso contrário
df['regular'] = np.where(
    df['phantom_billing'] | df['upcoding'] | df['duplicidade_pagamento'],
    'não',  # pelo menos 1 fraude
    'sim'   # sem fraudes
)

# Exibir amostra
print("Amostra de dados gerados com fraudes:")
print(df.head())

# Salvar e baixar CSV no Google Colab
csv_filename = "banco_dados_sintetico_operadora.csv"
df.to_csv(csv_filename, index=False)

from google.colab import files
files.download(csv_filename)

# Exibir exemplo das colunas de fraude + coluna regular
print("\nAmostra de 10 primeiros registros com colunas de fraude e coluna 'regular':")
print(df[['id_atendimento', 'phantom_billing', 'upcoding', 'duplicidade_pagamento', 'regular']].head(10))

# Exibir distribuição da coluna regular
print("\nDistribuição da variável 'regular':")
print(df['regular'].value_counts())

Amostra de dados gerados com fraudes:
   id_atendimento  beneficiario_id  prestador_id data_solicitacao CID_  \
0               1             8270           464       2024-03-06  H10   
1               2             1860           676       2023-07-23  G40   
2               3             6390           425       2023-03-16  H10   
3               4             6191           680       2023-04-23  B20   
4               5             6734           732       2024-05-05  C34   

   status_saude tipo_atendimento  procedimento_codigo autorizacao_status  \
0             1            exame                47030           aprovado   
1             0         consulta                41792           pendente   
2            -1         consulta                17388             negado   
3             0         cirurgia                65697           aprovado   
4             0         consulta                50224           pendente   

  data_realizacao   valor_pago procedimentos_urgencia  phant

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Amostra de 10 primeiros registros com colunas de fraude e coluna 'regular':
   id_atendimento  phantom_billing  upcoding  duplicidade_pagamento regular
0               1            False     False                  False     sim
1               2            False     False                   True     não
2               3             True     False                  False     não
3               4            False     False                  False     sim
4               5            False     False                  False     sim
5               6            False     False                  False     sim
6               7            False     False                  False     sim
7               8            False     False                  False     sim
8               9            False     False                  False     sim
9              10            False     False                  False     sim

Distribuição da variável 'regular':
regular
sim    9101
não     899
Name: count, dtype